## Basics of Qdrant

#### Ways to Get started with Qdrant Vectordatabase

-- In Memory: For quick tests or experiments, you might want to keep all the data only in memory. This way, the data is automatically deleted when the client is closed.

```bash
pip install qdrant
```

In [22]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

qdrant_client = QdrantClient(":memory:")

qdrant_client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)


True

On Disk: In local mode, without running a Qdrant server, your vectors can be saved to disk so they stay available between runs. Note: doesn't support multiple users at the same time.

In [ ]:
##No concurrent access allowed for this path
qdrant_client = QdrantClient(path="/tmp/langchain_qdrant")

qdrant_client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

On-Premise Server Deployment (for concurrency): To handle multiple users at once, you can run Qdrant locally using a Docker container.

Make sure docker is installed in system and is running:
- After installing Docker, run `docker ps` <br>
- If Docker is installed correctly, the command will work without errors and show something like:<br>
`CONTAINER ID   IMAGE           COMMAND             CREATED        STATUS        PORTS` <br>
This means Docker is running and ready to use.

```bash
docker pull qdrant/qdrant
docker run -p 6333:6333 -p 6334:6334 \
    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant
```

After that, You can access Qdrant on following url in local:

- **REST API** :  `localhost:6333`
- **Web UI** :  `localhost:6333/dashboard`
- **GRPC API** :  `localhost:6334`

Creating collection with vectorsize as 1024 and Distance function used to compare vectors

- COSINE = 'Cosine'
- DOT = 'Dot'
- EUCLID = 'Euclid'
- MANHATTAN = 'Manhattan'

In [ ]:
##Initialize client
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

qdrant_client = QdrantClient(url="http://localhost:6333")

qdrant_client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

True

#### Qdrant Collection
A Qdrant collection is like a folder or container where your vector data is stored and organized.
Each collection holds:
- Vectors – numerical representations of your data (like text, images, etc.)
- Payloads – extra information (like tags or metadata) linked to each vector
- Configuration – settings like vector size, distance metric (e.g., cosine or dot product), etc.

You can create multiple collections in Qdrant to separate different kinds of data or use cases.

#### 1. To check if a collection exists, use its name to look it up.

In [28]:
##check collection exists
qdrant_client.collection_exists(collection_name="demo_collection")

True

#### 2. Get details of collection

In [29]:
qdrant_client.get_collection(collection_name="demo_collection")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quan

The following color statuses are possible:

- 🟢 green: collection is ready
- 🟡 yellow: collection is optimizing
- ⚫ grey: collection is pending optimization (help)
- 🔴 red: an error occurred which the engine could not recover from

Right now its green so our collection  is ready

#### 3. Delete collection

In [ ]:
##To delete qdrant collection
qdrant_client.delete_collection(collection_name="demo_collection")

True

#### 4. List All Collection

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

qdrant_client = QdrantClient(url="http://localhost:6333")

qdrant_client.create_collection(
    collection_name="demo_collection1",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

qdrant_client.create_collection(
    collection_name="demo_collection2",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

True

In [35]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='demo_collection2'), CollectionDescription(name='demo_collection1'), CollectionDescription(name='projects_summary_aws_titan'), CollectionDescription(name='project_development_training_doc_db'), CollectionDescription(name='my_documents')])

#### 5. Adding Data (Metadata as well as original contents vectors) as payload 

In [36]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


qdrant_client = QdrantClient(":memory:")

qdrant_client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=3, distance=Distance.COSINE),
)


True

In [37]:
from qdrant_client.http import models

qdrant_client.upsert(
    collection_name="demo_collection",
    points=[
        models.PointStruct(
            id=1,
            payload={
                "color": "red",
            },
            vector=[0.9, 0.1, 0.1],
        ),
        models.PointStruct(
            id=2,
            payload={
                "color": "green",
            },
            vector=[0.1, 0.9, 0.1],
        ),
        models.PointStruct(
            id=3,
            payload={
                "color": "blue",
            },
            vector=[0.1, 0.1, 0.9],
        ),
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

#### 6. Retrieving content based on ids

In [38]:
qdrant_client.retrieve(
    collection_name="demo_collection",
    ids=[2],
)

[Record(id=2, payload={'color': 'green'}, vector=None, shard_key=None, order_value=None)]

#### 7. Deleting contents based on ids

In [39]:
qdrant_client.delete(
    collection_name="demo_collection",
    points_selector=models.PointIdsList(
        points=[2],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

#### 8. Retrieving content based on ids

In [40]:
qdrant_client.retrieve(
    collection_name="demo_collection",
    ids=[2],
)

[]

#### 9. Retrieving content based on key-value of payload

In [41]:
qdrant_client.scroll(
    collection_name="demo_collection",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="color", match=models.MatchValue(value="blue")),
        ]
    ),
    limit=1,
    with_payload=True,
    with_vectors=False,
)

([Record(id=3, payload={'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)